# LSTM Classification with MPQA Dataset
<hr>

We will build a text classification model using LSTM model on the MPQA Dataset. Since there is no standard train/test split for this dataset, we will use 10-Fold Cross Validation (CV). 

## Load the library

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
import random
# from nltk.tokenize import TweetTokenizer
from sklearn.model_selection import KFold

%config IPCompleter.greedy=True
%config IPCompleter.use_jedi=False
# nltk.download('twitter_samples')

In [2]:
tf.config.list_physical_devices('GPU') 

[]

## Load the Dataset

In [3]:
corpus = pd.read_pickle('../../../0_data/MPQA/MPQA.pkl')
corpus.label = corpus.label.astype(int)
print(corpus.shape)
corpus

(10606, 3)


,sentence,label,split
0,complaining,0,train
1,failing to support,0,train
2,desperately needs,0,train
3,many years of decay,0,train
4,no quick fix,0,train
...,...,...,...
10601,urged,1,train
10602,strictly abide,1,train
10603,hope,1,train
10604,strictly abide,1,train


In [4]:
corpus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10606 entries, 0 to 10605
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sentence  10606 non-null  object
 1   label     10606 non-null  int32 
 2   split     10606 non-null  object
dtypes: int32(1), object(2)
memory usage: 207.3+ KB


In [5]:
corpus.groupby( by='label').count()

,sentence,split
label,,
0,7294,7294
1,3312,3312


In [6]:
# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

In [7]:
sentences[0]

'complaining'

<!--## Split Dataset-->

# Data Preprocessing
<hr>

Preparing data for word embedding, especially for pre-trained word embedding like Word2Vec or GloVe, __don't use standard preprocessing steps like stemming or stopword removal__. Compared to our approach on cleaning the text when doing word count based feature extraction (e.g. TFIDF) such as removing stopwords, stemming etc, now we will keep these words as we do not want to lose such information that might help the model learn better.

__Tomas Mikolov__, one of the developers of Word2Vec, in _word2vec-toolkit: google groups thread., 2015_, suggests only very minimal text cleaning is required when learning a word embedding model. Sometimes, it's good to disconnect
In short, what we will do is:
- Puntuations removal
- Lower the letter case
- Tokenization

The process above will be handled by __Tokenizer__ class in TensorFlow

- <b>One way to choose the maximum sequence length is to just pick the length of the longest sentence in the training set.</b>

In [8]:
# Define a function to compute the max length of sequence
def max_length(sequences):
    '''
    input:
        sequences: a 2D list of integer sequences
    output:
        max_length: the max length of the sequences
    '''
    max_length = 0
    for i, seq in enumerate(sequences):
        length = len(seq)
        if max_length < length:
            max_length = length
    return max_length

In [9]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"

print("Example of sentence: ", sentences[4])

# Cleaning and Tokenization
tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(sentences)

# Turn the text into sequence
training_sequences = tokenizer.texts_to_sequences(sentences)
max_len = max_length(training_sequences)

print('Into a sequence of int:', training_sequences[4])

# Pad the sequence to have the same size
training_padded = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
print('Into a padded sequence:', training_padded[4])

Example of sentence:  no quick fix
Into a sequence of int: [25, 945, 1476]
Into a padded sequence: [  25  945 1476    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0]


In [10]:
word_index = tokenizer.word_index
# See the first 10 words in the vocabulary
for i, word in enumerate(word_index):
    print(word, word_index.get(word))
    if i==9:
        break
vocab_size = len(word_index)+1
print(vocab_size)

<UNK> 1
the 2
of 3
to 4
a 5
and 6
not 7
is 8
in 9
be 10
6236


# Model 1: Embedding Random
<hr>

<img src="model.png" style="width:700px;height:400px;"> <br>

## LSTM Model

In [11]:
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm

def define_model(input_dim = None, output_dim=300, max_length = None ):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(input_dim=input_dim, 
                                  mask_zero= True,
                                  output_dim=output_dim, 
                                  input_length=max_length, 
                                  input_shape=(max_length, )),
        
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=False)),
#         tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(units=1, activation='sigmoid')
    ])
    
    model.compile( loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
#     model.summary()
    return model

In [12]:
model_0 = define_model( input_dim=1000, max_length=100)
model_0.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 300)          300000    
_________________________________________________________________
bidirectional (Bidirectional (None, 100, 128)          186880    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 128)               98816     
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 585,825
Trainable params: 585,825
Non-trainable params: 0
_________________________________________________________________


In [17]:
class myCallback(tf.keras.callbacks.Callback):
    # Overide the method on_epoch_end() for our benefit
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') > 0.93):
            print("\nReached 93% accuracy so cancelling training!")
            self.model.stop_training=True


callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=10, verbose=2, 
                                             mode='auto', restore_best_weights=True)

## Train and Test the Model

In [18]:
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"

columns = ['acc1', 'acc2', 'acc3', 'acc4', 'acc5', 'acc6', 'acc7', 'acc8', 'acc9', 'acc10', 'AVG']
record = pd.DataFrame(columns = columns)

# prepare cross validation with 10 splits and shuffle = True
kfold = KFold(10, True)

# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

exp=0

# kfold.split() will return set indices for each split
acc_list = []
for train, test in kfold.split(sentences):
    
    exp+=1
    print('Training {}: '.format(exp))
    
    train_x, test_x = [], []
    train_y, test_y = [], []

    for i in train:
        train_x.append(sentences[i])
        train_y.append(labels[i])

    for i in test:
        test_x.append(sentences[i])
        test_y.append(labels[i])

    # Turn the labels into a numpy array
    train_y = np.array(train_y)
    test_y = np.array(test_y)

    # encode data using
    # Cleaning and Tokenization
    tokenizer = Tokenizer(oov_token=oov_tok)
    tokenizer.fit_on_texts(train_x)

    # Turn the text into sequence
    training_sequences = tokenizer.texts_to_sequences(train_x)
    test_sequences = tokenizer.texts_to_sequences(test_x)

    max_len = max_length(training_sequences)

    # Pad the sequence to have the same size
    Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
    Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

    word_index = tokenizer.word_index
    vocab_size = len(word_index)+1

    # Define the input shape
    model = define_model(input_dim=vocab_size, max_length=max_len)

    # Train the model
    model.fit(Xtrain, train_y, batch_size=32, epochs=40, verbose=1, 
              callbacks=[callbacks], validation_data=(Xtest, test_y))

    # evaluate the model
    loss, acc = model.evaluate(Xtest, test_y, verbose=0)
    print('Test Accuracy: {}'.format(acc*100))

    acc_list.append(acc*100)

mean_acc = np.array(acc_list).mean()
entries = acc_list + [mean_acc]

temp = pd.DataFrame([entries], columns=columns)
record = record.append(temp, ignore_index=True)
print()
print(record)
print()

Training 1: 
Epoch 1/40
299/299 [==============================] - 109s 290ms/step - loss: 0.5525 - accuracy: 0.7437 - val_loss: 0.3866 - val_accuracy: 0.8303
Epoch 2/40
299/299 [==============================] - 72s 242ms/step - loss: 0.1998 - accuracy: 0.9338 - val_loss: 0.3882 - val_accuracy: 0.8426
Epoch 3/40
299/299 [==============================] - 70s 233ms/step - loss: 0.1369 - accuracy: 0.9524 - val_loss: 0.4199 - val_accuracy: 0.8445
Epoch 4/40
299/299 [==============================] - 68s 227ms/step - loss: 0.1041 - accuracy: 0.9646 - val_loss: 0.5014 - val_accuracy: 0.8407
Epoch 5/40
299/299 [==============================] - 70s 234ms/step - loss: 0.0834 - accuracy: 0.9716 - val_loss: 0.5926 - val_accuracy: 0.8435
Epoch 6/40
299/299 [==============================] - 69s 230ms/step - loss: 0.0602 - accuracy: 0.9757 - val_loss: 0.6757 - val_accuracy: 0.8398
Epoch 7/40
299/299 [==============================] - 69s 232ms/step - loss: 0.0481 - accuracy: 0.9804 - val_loss: 0

299/299 [==============================] - 105s 350ms/step - loss: 0.0892 - accuracy: 0.9703 - val_loss: 0.5188 - val_accuracy: 0.8549
Epoch 6/40
299/299 [==============================] - 104s 348ms/step - loss: 0.0682 - accuracy: 0.9758 - val_loss: 0.5637 - val_accuracy: 0.8558
Epoch 7/40
299/299 [==============================] - 104s 348ms/step - loss: 0.0480 - accuracy: 0.9799 - val_loss: 0.6639 - val_accuracy: 0.8492
Epoch 8/40
299/299 [==============================] - 103s 346ms/step - loss: 0.0447 - accuracy: 0.9803 - val_loss: 0.7027 - val_accuracy: 0.8454
Epoch 9/40
299/299 [==============================] - 105s 350ms/step - loss: 0.0369 - accuracy: 0.9831 - val_loss: 0.7941 - val_accuracy: 0.8501
Epoch 10/40
299/299 [==============================] - 103s 346ms/step - loss: 0.0363 - accuracy: 0.9828 - val_loss: 0.7940 - val_accuracy: 0.8501
Epoch 11/40
299/299 [==============================] - 102s 341ms/step - loss: 0.0346 - accuracy: 0.9845 - val_loss: 0.8739 - val_accu

299/299 [==============================] - 119s 399ms/step - loss: 0.2065 - accuracy: 0.9297 - val_loss: 0.3972 - val_accuracy: 0.8481
Epoch 3/40
299/299 [==============================] - 115s 384ms/step - loss: 0.1368 - accuracy: 0.9531 - val_loss: 0.4553 - val_accuracy: 0.8406
Epoch 4/40
299/299 [==============================] - 114s 380ms/step - loss: 0.1094 - accuracy: 0.9639 - val_loss: 0.4892 - val_accuracy: 0.8462
Epoch 5/40
299/299 [==============================] - 113s 379ms/step - loss: 0.0854 - accuracy: 0.9687 - val_loss: 0.5569 - val_accuracy: 0.8415
Epoch 6/40
299/299 [==============================] - 114s 381ms/step - loss: 0.0682 - accuracy: 0.9730 - val_loss: 0.7185 - val_accuracy: 0.8358
Epoch 7/40
299/299 [==============================] - 113s 379ms/step - loss: 0.0525 - accuracy: 0.9775 - val_loss: 0.7469 - val_accuracy: 0.8387
Epoch 8/40
299/299 [==============================] - 113s 379ms/step - loss: 0.0366 - accuracy: 0.9849 - val_loss: 0.8872 - val_accura

## Summary

In [19]:
record

,acc1,acc2,acc3,acc4,acc5,acc6,acc7,acc8,acc9,acc10,AVG
0,84.448636,86.145145,86.804903,85.20264,86.427897,86.616397,86.320752,86.415094,85.849059,86.603773,86.083429


In [20]:
report = record
report = report.to_excel('LSTM_MPQA_v2.xlsx', sheet_name='random')

# Model 2: Word2Vec Static

__Using and updating pre-trained embeddings__
* In this part, we will create an Embedding layer in Tensorflow Keras using a pre-trained word embedding called Word2Vec 300-d tht has been trained 100 bilion words from Google News.
* In this part,  we will leave the embeddings fixed instead of updating them (dynamic).

1. __Load `Word2Vec` Pre-trained Word Embedding__

In [21]:
from gensim.models import KeyedVectors
word2vec = KeyedVectors.load_word2vec_format('../GoogleNews-vectors-negative300.bin', binary=True)

In [22]:
# Access the dense vector value for the word 'handsome'
# word2vec.word_vec('handsome') # 0.11376953
word2vec.word_vec('cool') # 1.64062500e-01

array([ 1.64062500e-01,  1.87500000e-01, -4.10156250e-02,  1.25000000e-01,
       -3.22265625e-02,  8.69140625e-02,  1.19140625e-01, -1.26953125e-01,
        1.77001953e-02,  8.83789062e-02,  2.12402344e-02, -2.00195312e-01,
        4.83398438e-02, -1.01074219e-01, -1.89453125e-01,  2.30712891e-02,
        1.17675781e-01,  7.51953125e-02, -8.39843750e-02, -1.33666992e-02,
        1.53320312e-01,  4.08203125e-01,  3.80859375e-02,  3.36914062e-02,
       -4.02832031e-02, -6.88476562e-02,  9.03320312e-02,  2.12890625e-01,
        1.72119141e-02, -6.44531250e-02, -1.29882812e-01,  1.40625000e-01,
        2.38281250e-01,  1.37695312e-01, -1.76757812e-01, -2.71484375e-01,
       -1.36718750e-01, -1.69921875e-01, -9.15527344e-03,  3.47656250e-01,
        2.22656250e-01, -3.06640625e-01,  1.98242188e-01,  1.33789062e-01,
       -4.34570312e-02, -5.12695312e-02, -3.46679688e-02, -8.49609375e-02,
        1.01562500e-01,  1.42578125e-01, -7.95898438e-02,  1.78710938e-01,
        2.30468750e-01,  

2. __Check number of training words present in Word2Vec__

In [23]:
def training_words_in_word2vector(word_to_vec_map, word_to_index):
    '''
    input:
        word_to_vec_map: a word2vec GoogleNews-vectors-negative300.bin model loaded using gensim.models
        word_to_index: word to index mapping from training set
    '''
    
    vocab_size = len(word_to_index) + 1
    count = 0
    # Set each row "idx" of the embedding matrix to be 
    # the word vector representation of the idx'th word of the vocabulary
    for word, idx in word_to_index.items():
        if word in word_to_vec_map:
            count+=1
            
    return print('Found {} words present from {} training vocabulary in the set of pre-trained word vector'.format(count, vocab_size))

In [24]:
# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

# Cleaning and Tokenization
tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(sentences)

word_index = tokenizer.word_index
training_words_in_word2vector(word2vec, word_index)

Found 6083 words present from 6236 training vocabulary in the set of pre-trained word vector


2. __Define a `pretrained_embedding_layer` function__

In [25]:
from tensorflow.keras.layers import Embedding

def pretrained_embedding_matrix(word_to_vec_map, word_to_index):
    '''
    input:
        word_to_vec_map: a word2vec GoogleNews-vectors-negative300.bin model loaded using gensim.models
        word_to_index: word to index mapping from training set
    '''
    
    # adding 1 to fit Keras embedding (requirement)
    vocab_size = len(word_to_index) + 1
    # define dimensionality of your pre-trained word vectors (= 300)
    emb_dim = word_to_vec_map.word_vec('handsome').shape[0]
    
    
    embed_matrix = np.zeros((vocab_size, emb_dim))
    
    # Set each row "idx" of the embedding matrix to be 
    # the word vector representation of the idx'th word of the vocabulary
    for word, idx in word_to_index.items():
        if word in word_to_vec_map:
            embed_matrix[idx] = word_to_vec_map.word_vec(word)
            
        # initialize the unknown word with standard normal distribution values
        else:
            embed_matrix[idx] = np.random.randn(emb_dim)
            
    return embed_matrix

In [26]:
# Test the function
w_2_i = {'<UNK>': 1, 'handsome': 2, 'cool': 3, 'shit': 4 }
em_matrix = pretrained_embedding_matrix(word2vec, w_2_i)
em_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-1.39502923, -0.19060103,  0.92061864, ..., -0.45392575,
        -1.18848863, -1.29073577],
       [ 0.11376953,  0.1796875 , -0.265625  , ..., -0.21875   ,
        -0.03930664,  0.20996094],
       [ 0.1640625 ,  0.1875    , -0.04101562, ...,  0.10888672,
        -0.01019287,  0.02075195],
       [ 0.10888672, -0.16699219,  0.08984375, ..., -0.19628906,
        -0.23144531,  0.04614258]])

## LSTM Model

In [27]:
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm

def define_model_2(input_dim = None, output_dim=300, max_length = None, emb_matrix=None):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(input_dim=input_dim, 
                                  mask_zero= True,
                                  output_dim=output_dim, 
                                  input_length=max_length, 
                                  input_shape=(max_length, ),
                                  # Assign the embedding weight with word2vec embedding marix
                                  weights = [emb_matrix],
                                  # Set the weight to be not trainable (static)
                                  trainable = False),
        
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=False)),
#         tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(units=1, activation='sigmoid')
    ])
    
    model.compile( loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
#     model.summary()
    return model

In [28]:
model_0 = define_model_2( input_dim=1000, max_length=100, emb_matrix=np.random.rand(1000, 300))
model_0.summary()

Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_19 (Embedding)     (None, 100, 300)          300000    
_________________________________________________________________
bidirectional_38 (Bidirectio (None, 100, 128)          186880    
_________________________________________________________________
bidirectional_39 (Bidirectio (None, 128)               98816     
_________________________________________________________________
dropout_19 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_19 (Dense)             (None, 1)                 129       
Total params: 585,825
Trainable params: 285,825
Non-trainable params: 300,000
_________________________________________________________________


## Train and Test the Model

In [29]:
class myCallback(tf.keras.callbacks.Callback):
    # Overide the method on_epoch_end() for our benefit
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') >= 0.9):
            print("\nReached 90% accuracy so cancelling training!")
            self.model.stop_training=True

callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=10, verbose=2, 
                                             mode='auto', restore_best_weights=True)

In [30]:
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"

columns = ['acc1', 'acc2', 'acc3', 'acc4', 'acc5', 'acc6', 'acc7', 'acc8', 'acc9', 'acc10', 'AVG']
record2 = pd.DataFrame(columns = columns)

# prepare cross validation with 10 splits and shuffle = True
kfold = KFold(10, True)

# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

exp=0

# kfold.split() will return set indices for each split
acc_list = []
for train, test in kfold.split(sentences):
    
    exp+=1
    print('Training {}: '.format(exp))
    
    train_x, test_x = [], []
    train_y, test_y = [], []

    for i in train:
        train_x.append(sentences[i])
        train_y.append(labels[i])

    for i in test:
        test_x.append(sentences[i])
        test_y.append(labels[i])

    # Turn the labels into a numpy array
    train_y = np.array(train_y)
    test_y = np.array(test_y)

    # encode data using
    # Cleaning and Tokenization
    tokenizer = Tokenizer(oov_token=oov_tok)
    tokenizer.fit_on_texts(train_x)

    # Turn the text into sequence
    training_sequences = tokenizer.texts_to_sequences(train_x)
    test_sequences = tokenizer.texts_to_sequences(test_x)

    max_len = max_length(training_sequences)

    # Pad the sequence to have the same size
    Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
    Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

    word_index = tokenizer.word_index
    vocab_size = len(word_index)+1
    
    emb_matrix = pretrained_embedding_matrix(word2vec, word_index)

    # Define the input shape
    model = define_model_2(input_dim=vocab_size, max_length=max_len, emb_matrix=emb_matrix)

    # Train the model
    model.fit(Xtrain, train_y, batch_size=32, epochs=40, verbose=1, 
              callbacks=[callbacks], validation_data=(Xtest, test_y))

    # evaluate the model
    loss, acc = model.evaluate(Xtest, test_y, verbose=0)
    print('Test Accuracy: {}'.format(acc*100))

    acc_list.append(acc*100)

mean_acc = np.array(acc_list).mean()
entries = acc_list + [mean_acc]

temp = pd.DataFrame([entries], columns=columns)
record2 = record2.append(temp, ignore_index=True)
print()
print(record2)
print()

Training 1: 
Epoch 1/40
299/299 [==============================] - 160s 448ms/step - loss: 0.4935 - accuracy: 0.7606 - val_loss: 0.3906 - val_accuracy: 0.8106
Epoch 2/40
299/299 [==============================] - 108s 361ms/step - loss: 0.2812 - accuracy: 0.8849 - val_loss: 0.4433 - val_accuracy: 0.8115
Epoch 3/40
299/299 [==============================] - 108s 360ms/step - loss: 0.2592 - accuracy: 0.8983 - val_loss: 0.4075 - val_accuracy: 0.8238
Epoch 4/40
299/299 [==============================] - 110s 368ms/step - loss: 0.2204 - accuracy: 0.9160 - val_loss: 0.3392 - val_accuracy: 0.8360
Epoch 5/40
299/299 [==============================] - 110s 368ms/step - loss: 0.2046 - accuracy: 0.9203 - val_loss: 0.3783 - val_accuracy: 0.8285
Epoch 6/40
299/299 [==============================] - 109s 366ms/step - loss: 0.1856 - accuracy: 0.9293 - val_loss: 0.3304 - val_accuracy: 0.8709
Epoch 7/40
299/299 [==============================] - 110s 368ms/step - loss: 0.1583 - accuracy: 0.9362 - val_l

299/299 [==============================] - 110s 368ms/step - loss: 0.0630 - accuracy: 0.9791 - val_loss: 0.5001 - val_accuracy: 0.8737
Epoch 15/40
299/299 [==============================] - 111s 371ms/step - loss: 0.0636 - accuracy: 0.9762 - val_loss: 0.5241 - val_accuracy: 0.8690
Epoch 16/40
299/299 [==============================] - 112s 375ms/step - loss: 0.0555 - accuracy: 0.9783 - val_loss: 0.5509 - val_accuracy: 0.8765
Epoch 17/40
299/299 [==============================] - 111s 372ms/step - loss: 0.0516 - accuracy: 0.9783 - val_loss: 0.5962 - val_accuracy: 0.8652
Epoch 18/40
299/299 [==============================] - 112s 373ms/step - loss: 0.0407 - accuracy: 0.9849 - val_loss: 0.6120 - val_accuracy: 0.8756
Epoch 19/40
299/299 [==============================] - 110s 366ms/step - loss: 0.0487 - accuracy: 0.9817 - val_loss: 0.5694 - val_accuracy: 0.8709
Epoch 20/40
299/299 [==============================] - 107s 358ms/step - loss: 0.0550 - accuracy: 0.9776 - val_loss: 0.5341 - val_

299/299 [==============================] - 97s 323ms/step - loss: 0.2137 - accuracy: 0.9211 - val_loss: 0.3808 - val_accuracy: 0.8209
Epoch 6/40
299/299 [==============================] - 97s 324ms/step - loss: 0.1816 - accuracy: 0.9310 - val_loss: 0.4247 - val_accuracy: 0.8002
Epoch 7/40
299/299 [==============================] - 97s 323ms/step - loss: 0.1666 - accuracy: 0.9323 - val_loss: 0.4251 - val_accuracy: 0.8068
Epoch 8/40
299/299 [==============================] - 96s 322ms/step - loss: 0.1524 - accuracy: 0.9366 - val_loss: 0.4859 - val_accuracy: 0.8011
Epoch 9/40
299/299 [==============================] - 96s 323ms/step - loss: 0.1358 - accuracy: 0.9449 - val_loss: 0.4854 - val_accuracy: 0.8143
Epoch 10/40
299/299 [==============================] - 96s 323ms/step - loss: 0.1134 - accuracy: 0.9543 - val_loss: 0.4869 - val_accuracy: 0.8049
Epoch 11/40
299/299 [==============================] - 96s 322ms/step - loss: 0.1017 - accuracy: 0.9573 - val_loss: 0.6024 - val_accuracy: 0

299/299 [==============================] - 137s 459ms/step - loss: 0.1003 - accuracy: 0.9660 - val_loss: 0.5561 - val_accuracy: 0.8472
Epoch 13/40
299/299 [==============================] - 138s 462ms/step - loss: 0.0815 - accuracy: 0.9698 - val_loss: 0.7286 - val_accuracy: 0.8377
Epoch 14/40
299/299 [==============================] - 137s 459ms/step - loss: 0.0694 - accuracy: 0.9731 - val_loss: 0.7153 - val_accuracy: 0.8547
Epoch 15/40
299/299 [==============================] - 138s 461ms/step - loss: 0.0636 - accuracy: 0.9752 - val_loss: 0.8168 - val_accuracy: 0.8358
Epoch 16/40
299/299 [==============================] - 137s 460ms/step - loss: 0.0583 - accuracy: 0.9770 - val_loss: 0.9211 - val_accuracy: 0.8245
Epoch 17/40
299/299 [==============================] - 137s 459ms/step - loss: 0.0565 - accuracy: 0.9781 - val_loss: 0.7825 - val_accuracy: 0.8443
Epoch 18/40
299/299 [==============================] - 137s 459ms/step - loss: 0.0448 - accuracy: 0.9817 - val_loss: 0.8303 - val_

## Summary

In [31]:
record2

,acc1,acc2,acc3,acc4,acc5,acc6,acc7,acc8,acc9,acc10,AVG
0,88.689917,82.563621,87.65316,80.395854,85.768145,89.16117,85.66038,85.566038,87.452829,87.26415,86.017526


In [32]:
report = record2
report = report.to_excel('LSTM_MPQA_v2_2.xlsx', sheet_name='static')

# Model 3: Word2Vec - Dynamic

* In this part,  we will fine tune the embeddings while training (dynamic).

## LSTM Model

In [33]:
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm

def define_model_3(input_dim = None, output_dim=300, max_length = None, emb_matrix=None):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(input_dim=input_dim, 
                                  mask_zero= True,
                                  output_dim=output_dim, 
                                  input_length=max_length, 
                                  input_shape=(max_length, ),
                                  # Assign the embedding weight with word2vec embedding marix
                                  weights = [emb_matrix],
                                  # Set the weight to be not trainable (static)
                                  trainable = True),
        
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=False)),
#         tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(units=1, activation='sigmoid')
    ])
    
    model.compile( loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
#     model.summary()
    return model

In [34]:
model_0 = define_model_3( input_dim=1000, max_length=100, emb_matrix=np.random.rand(1000, 300))
model_0.summary()

Model: "sequential_30"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_30 (Embedding)     (None, 100, 300)          300000    
_________________________________________________________________
bidirectional_60 (Bidirectio (None, 100, 128)          186880    
_________________________________________________________________
bidirectional_61 (Bidirectio (None, 128)               98816     
_________________________________________________________________
dropout_30 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_30 (Dense)             (None, 1)                 129       
Total params: 585,825
Trainable params: 585,825
Non-trainable params: 0
_________________________________________________________________


## Train and Test the Model

In [35]:
class myCallback(tf.keras.callbacks.Callback):
    # Overide the method on_epoch_end() for our benefit
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') > 0.93):
            print("\nReached 93% accuracy so cancelling training!")
            self.model.stop_training=True

callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=10, verbose=2, 
                                             mode='auto', restore_best_weights=True)

In [36]:
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"

columns = ['acc1', 'acc2', 'acc3', 'acc4', 'acc5', 'acc6', 'acc7', 'acc8', 'acc9', 'acc10', 'AVG']
record3 = pd.DataFrame(columns = columns)

# prepare cross validation with 10 splits and shuffle = True
kfold = KFold(10, True)

# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

exp=0

# kfold.split() will return set indices for each split
acc_list = []
for train, test in kfold.split(sentences):
    
    exp+=1
    print('Training {}: '.format(exp))
    
    train_x, test_x = [], []
    train_y, test_y = [], []

    for i in train:
        train_x.append(sentences[i])
        train_y.append(labels[i])

    for i in test:
        test_x.append(sentences[i])
        test_y.append(labels[i])

    # Turn the labels into a numpy array
    train_y = np.array(train_y)
    test_y = np.array(test_y)

    # encode data using
    # Cleaning and Tokenization
    tokenizer = Tokenizer(oov_token=oov_tok)
    tokenizer.fit_on_texts(train_x)

    # Turn the text into sequence
    training_sequences = tokenizer.texts_to_sequences(train_x)
    test_sequences = tokenizer.texts_to_sequences(test_x)

    max_len = max_length(training_sequences)

    # Pad the sequence to have the same size
    Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
    Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

    word_index = tokenizer.word_index
    vocab_size = len(word_index)+1
    
    emb_matrix = pretrained_embedding_matrix(word2vec, word_index)

    # Define the input shape
    model = define_model_3(input_dim=vocab_size, max_length=max_len, emb_matrix=emb_matrix)

    # Train the model
    model.fit(Xtrain, train_y, batch_size=32, epochs=40, verbose=1, 
              callbacks=[callbacks], validation_data=(Xtest, test_y))

    # evaluate the model
    loss, acc = model.evaluate(Xtest, test_y, verbose=0)
    print('Test Accuracy: {}'.format(acc*100))

    acc_list.append(acc*100)

mean_acc = np.array(acc_list).mean()
entries = acc_list + [mean_acc]

temp = pd.DataFrame([entries], columns=columns)
record3 = record3.append(temp, ignore_index=True)
print()
print(record)
print()

Training 1: 
Epoch 1/40
299/299 [==============================] - 191s 552ms/step - loss: 0.4807 - accuracy: 0.7738 - val_loss: 0.2875 - val_accuracy: 0.8822
Epoch 2/40
299/299 [==============================] - 157s 526ms/step - loss: 0.1876 - accuracy: 0.9364 - val_loss: 0.2976 - val_accuracy: 0.8841
Epoch 3/40
299/299 [==============================] - 149s 500ms/step - loss: 0.1378 - accuracy: 0.9555 - val_loss: 0.3297 - val_accuracy: 0.8878
Epoch 4/40
299/299 [==============================] - 148s 494ms/step - loss: 0.0981 - accuracy: 0.9654 - val_loss: 0.3382 - val_accuracy: 0.8897
Epoch 5/40
299/299 [==============================] - 149s 498ms/step - loss: 0.0759 - accuracy: 0.9686 - val_loss: 0.4486 - val_accuracy: 0.8822
Epoch 6/40
299/299 [==============================] - 149s 498ms/step - loss: 0.0571 - accuracy: 0.9783 - val_loss: 0.5567 - val_accuracy: 0.8803
Epoch 7/40
299/299 [==============================] - 147s 493ms/step - loss: 0.0496 - accuracy: 0.9783 - val_l

299/299 [==============================] - 134s 448ms/step - loss: 0.1333 - accuracy: 0.9536 - val_loss: 0.5030 - val_accuracy: 0.8709
Epoch 4/40
299/299 [==============================] - 134s 448ms/step - loss: 0.0969 - accuracy: 0.9673 - val_loss: 0.5122 - val_accuracy: 0.8709
Epoch 5/40
299/299 [==============================] - 133s 443ms/step - loss: 0.0747 - accuracy: 0.9698 - val_loss: 0.5572 - val_accuracy: 0.8718
Epoch 6/40
299/299 [==============================] - 135s 450ms/step - loss: 0.0635 - accuracy: 0.9730 - val_loss: 0.7561 - val_accuracy: 0.8728
Epoch 7/40
299/299 [==============================] - 135s 453ms/step - loss: 0.0516 - accuracy: 0.9784 - val_loss: 0.7016 - val_accuracy: 0.8586
Epoch 8/40
299/299 [==============================] - 134s 449ms/step - loss: 0.0414 - accuracy: 0.9819 - val_loss: 0.7477 - val_accuracy: 0.8577
Epoch 9/40
299/299 [==============================] - 134s 449ms/step - loss: 0.0353 - accuracy: 0.9835 - val_loss: 0.9907 - val_accura

299/299 [==============================] - 188s 527ms/step - loss: 0.4748 - accuracy: 0.7820 - val_loss: 0.4687 - val_accuracy: 0.7991
Epoch 2/40
299/299 [==============================] - 152s 508ms/step - loss: 0.1941 - accuracy: 0.9309 - val_loss: 0.4357 - val_accuracy: 0.8047
Epoch 3/40
299/299 [==============================] - 146s 490ms/step - loss: 0.1312 - accuracy: 0.9555 - val_loss: 0.5537 - val_accuracy: 0.8038
Epoch 4/40
299/299 [==============================] - 144s 481ms/step - loss: 0.1066 - accuracy: 0.9614 - val_loss: 0.6355 - val_accuracy: 0.7943
Epoch 5/40
299/299 [==============================] - 143s 477ms/step - loss: 0.0769 - accuracy: 0.9690 - val_loss: 0.7197 - val_accuracy: 0.7858
Epoch 6/40
299/299 [==============================] - 142s 475ms/step - loss: 0.0563 - accuracy: 0.9767 - val_loss: 0.7540 - val_accuracy: 0.7840
Epoch 7/40
299/299 [==============================] - 142s 475ms/step - loss: 0.0528 - accuracy: 0.9770 - val_loss: 0.8263 - val_accura

## Summary

In [37]:
record3

,acc1,acc2,acc3,acc4,acc5,acc6,acc7,acc8,acc9,acc10,AVG
0,88.97267,87.935907,85.57964,87.370408,88.878417,87.84166,81.603771,80.4717,87.924528,84.150946,86.072965


In [38]:
report = record3
report = report.to_excel('LSTM_MPQA_v2_3.xlsx', sheet_name='dynamic')